### **5.16 - [Practica] Running your DAGs with the Kubernetes Executor**

Now everything is set up, it’s time to finally run our DAGs with the Kubernetes Executor.  In this short video we are going to see what happens when tasks of a DAG are run through the Kubernetes  executor. From your web browser,  you should have Rancher with your Kubernetes Cluster and Airflow running.  If not please, check the previous video.  The web server, the metadata database as well as the scheduler should be active and running as shown  here.  Notice that you can check the pods running in your kubernetes cluster by clicking on “Resources”  then “Workloads”. Here we obtain the following three components of Airflow without an external queue like  it is the case with the CeleryExecutor, within this namespace.  All right,  if you remember, we have seen that we need a way to access the pods from the outside of the cluster  using services.  A service in Kubernetes is responsible for enabling network access to a set of pods.  There are different kinds of services as you will see in section 7, but the most basic type of services  is the cluster IP.  It makes the service only reachable from within the cluster.  That’s what you can see by clicking on “Service Discovery”. Those two services are Cluster IP services  and if you take a look below the web server service, you can see the cluster IP associated. This ip address  is only known inside the kubernetes cluster and so you can’t reach the web server by using this ip address  from your web browser.  That being said, how can we access the Airflow user interface from the outside of the cluster?  Well, the simplest way to do it is by using port forwarding.  Actually, if you click on “Apps” then “airflow”, and “Notes”, by scrolling down, you can see here, the instructions  to access Airflow from your web browser.  Let’s open a session to the master node in order to execute those instructions.  Open your terminal, and check that you are under the folder airflow-materials/airflow  -section-5/vagrant-files. Then execute the command “vagrant ssh master”.  Now you are connected, go back to Rancher and copy the following line to export the name of the POD running  your webserver  Then in your terminal, paste the command and hit enter.  Okay.  If you type “echo $POD_NAME”  you obtain the following name.  This name corresponds to the name of the POD running your web server.  You can check this by typing the command “kubectl  get pods  --all-  namespaces”.  As you can see here, you have the same name.  All right,  now go back to Rancher and copy the last line in order to create the port forwarding between your pod  and the node where the command will be executed. In your terminal, paste the command and just after  “kubectl port-forward”,  type “--address  0.0.0.0” so that all ip v4 addresses are forwarded. Enter.  Ok, now the port forwarding is active,  if you go to your web browser, open a new tab,  and type 192.168.  219.2:8080,  notice that this ip address corresponds to the ip address of the master node as defined in the Vagrant  file.  Enter. And it doesn’t work.  Why?  Because we forwarded the port 8080 between the pod and the master node of the kubernetes  cluster but not between the master node and the virtual machine. Since our cluster is running inside  VMs with virtualbox  we have one more step before accessing the Airflow UI. Open virtualbox,  here you have the three nodes of your kubernetes cluster. Right click on “master”. Then “settings”, “network”,  “advanced” and “port forwarding”. From there,  click on this icon to create a new entry and type 192.168.219  .2 in Host IP and Guest IP.  Then specify the port 8080 for both Host Port and Guest Port.  Now click on “Ok”  twice. Back to your web browser, refresh the page,  and we got the Airflow UI.  Perfect. As you can see here we only have one DAG called parallel_dag.  Why?  Because the repository marclamberti/airflow-dags cloned during the initialization of our pods  has only this DAG in it.  If you open a new tab and go to https://github.com  /marclamberti/airflow-dags,  you can see that only the DAG parallel_dag exists in the repository.  If you want to set your own repo you will need to change the value of the variable AIRFLOW  __KUBERNETES__GIT_REPO in the config map of the Helm chart corresponding  to Airflow.  If you don't know what I’m talking about, check the previous video. All right, before triggering the DAG  we are going to open a new tab in our terminal and open a session to the master node as we did previously  with the command “vagrant ssh master”.  Now to show you what’s going on when a DAG is triggered with the kubernetesExecutor, type the following  command  “watch kubectl  get pods  -n” and the namespace where Airflow is running that you can get from there.  Copy the value and paste it. Enter. Now you are able to watch in real time if a pod is created, terminated  and so on in the namespace where your Airflow instance is running. Here we have the three pods corresponding  to the database, scheduler and web server of Airflow.  Now from the Airflow UI, turn on the toggle of the DAG parallel_dag  and go back to your terminal.  You can see that three pods are being created corresponding to the first three tasks to execute in parallel  from the DAG. The names of the pods indicate that the tasks 1 and 2 and 3 from parallel_dag are  going to be executed.  Notice the status of the PODs. Here,  init means init-container as we have seen previously and so the pod is cloning the repository  containing the DAGs.  If we continue to wait,  at some point we will the PODs moving from init to podInitliazing, running and terminating.  So that’s it,  you are now able to run Airflow with the KubernetesExecutor and execute your tasks in your Kubernetes  cluster. Before moving forward  we are going to stop everything as the cluster consumes a lot of resources. Close this tab, and hit control-c  in order to stop the port forwarding. Then hit control-d to exit the session. From the vagrant-files folder,  type “vagrant destroy”  in order to destroy all the vms created. Confirm for each node.  Ok now type “docker ps”  and stop the docker container running Rancher by typing “docker stop”  then copy and paste the container id.  Enter. Finally type “docker rm“  with the same container id. Enter. And we are done. If your machine struggles to run everything,  don’t worry in the section 7 I will show you how to set up the same architecture but this time using  AWS EKS. All right, I hope you enjoyed this video, the KubernetesExecutor is a powerful executor  to use with its own complexity because of Kubernetes, nonetheless  if you feel your team is ready, I strongly encourage you to go with it in production. Don’t forget that  it is actually used by Bloomberg.  